In [1]:
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# import Action chains 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import regex as re
import time
import random

# create webdriver object
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
   
# create action chain object
action = ActionChains(driver)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\morroe1\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [2]:
# navigate to website
driver.get("https://www.aa.com/booking/find-flights")
time.sleep(random.uniform(3.072618165231841,5.9364916385))
# driver.get("https://www.southwest.com/")
# time.sleep(random.uniform(3.072618165231841,5.9364916385))

In [3]:
from_airport = "SNA"
to_airport = "DFW"

In [4]:
depart_date = "06/24/2022"
return_date = "06/26/2022"

In [5]:
def button_press(action):
    if action == "keystroke":
        return random.uniform(.111315816564,.29978136531)
    elif action == "enter_search":
        return random.uniform(34.6551845131843, 37.3189651876)
    else :
        return random.uniform(.3098741610684,.5098135130)

In [6]:
def set_time_of_day(element):
    
    # Select the element
    action.move_to_element(element).perform()
    time.sleep(button_press(0))
    action.click(element).perform()
    time.sleep(button_press(0))

#     # Move down to "Early Evening"
#     for x in range(0,4):
#         action.send_keys(Keys.DOWN)
#         time.sleep(button_press("keystroke"))
        
    # move up to "All Day"
    action.send_keys(Keys.UP)
    time.sleep(button_press("keystroke"))

    action.click(element).perform()
    time.sleep(button_press(0))

In [7]:
def enter_airport(airport,element):
    action.move_to_element(element).perform()
    time.sleep(button_press(0))

    action.click(element).perform()
    time.sleep(button_press(0))

    for x in range(0,3):
        action.send_keys(Keys.BACKSPACE).perform()
        time.sleep(button_press("keystroke"))

    for letter in airport:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))

    time.sleep(button_press(0))  

In [8]:
def enter_dates(date,element):
    action.move_to_element(element).perform()
    time.sleep(button_press(0))
    
    action.click(element).perform()
    time.sleep(button_press(0))

    for letter in date:
        action.send_keys(letter).perform()
        time.sleep(button_press("keystroke"))
        
    time.sleep(button_press(0))  

In [9]:
# Set Time of Day for Departure and Return
set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelTime"]'))
set_time_of_day(driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelTime"]'))

In [10]:
# Set the travel dates
enter_dates(depart_date,driver.find_element(by=By.XPATH,value='//*[@id="segments0.travelDate"]'))
enter_dates(return_date,driver.find_element(by=By.XPATH,value='//*[@id="segments1.travelDate"]'))

In [11]:
# Set the airports
enter_airport(from_airport,driver.find_element(by=By.XPATH,value=('//*[@id="segments0.origin"]')))
enter_airport(to_airport,driver.find_element(by=By.XPATH,value=('//*[@id="segments0.destination"]')))

In [12]:
# Hitting the ENTER key while still in the airports field will execute a search
action.send_keys(Keys.ENTER).perform()
time.sleep(button_press("enter_search"))

In [13]:
# Get the html from the page
html = driver.page_source
html_soup = soup(html,'html.parser')

In [14]:
# Identifying the list elements that contain flight and price information
list_elements = html_soup.find_all("li",id=re.compile("slice0Flight[0-9]+"))

# i = 0
# lowest_fare = 2**15
# lowest_fare_element = []
# for x in list_elements:
#     list_elements[i] = re.findall("<li.*?>",str(x.prettify()))[0]
#     new_lowest_fare = re.findall("data-lowestfare=\".*?\"",list_elements[i])[0]
#     new_lowest_fare = float(re.findall("[0-9]+",new_lowest_fare)[0])
#     if new_lowest_fare < lowest_fare:
#         lowest_fare = new_lowest_fare
#         lowest_fare_element.append(x)
# #     print(f"{i}: {list_elements[i]}")
# #     print(lowest_fare_element)
#     i += 1

# # Filter to select the shortest trip duration
# i = 0
# shortest_trip = 2**15
# for x in lowest_fare_element:
#     lowest_fare_element[i] = re.findall("<li.*?>",str(x.prettify()))[0]
#     new_shortest_trip = re.findall("data-triptime=\".*?\"",lowest_fare_element[i])[0]
#     new_shortest_trip = float(re.findall("[0-9]+",new_shortest_trip)[0])
#     if new_shortest_trip < shortest_trip:
#         shortest_trip = new_shortest_trip
# #     print(lowest_fare_element[i])
#     print(shortest_trip)
    
i = 0
lowest_fare = 2**15
for x in list_elements:
    x = re.findall("<li.*?>",str(x.prettify()))[0]
    new_lowest_fare = re.findall("data-lowestfare=\".*?\"",x)[0]
    new_lowest_fare = float(re.findall("[0-9]+",new_lowest_fare)[0])
    if new_lowest_fare < lowest_fare:
        lowest_fare = new_lowest_fare
        lowest_fare_element = list_elements[i]
#     print(f"{i}: {list_elements[i]}")
    i += 1
# print(lowest_fare_element)

lowest_fare_element = re.findall("<button.*?MainCabin.*?>",str(lowest_fare_element.prettify()))[0]
# print(lowest_fare_element)

lowest_fare_id = re.findall("id=\"slice\dFlight[0-9]+MainCabin\"",lowest_fare_element)[0]
# print(lowest_fare_id)
lowest_fare_id = f"//*[@{lowest_fare_id}]"
# print(lowest_fare_id)

lowest_fare_element = driver.find_element(
    by=By.XPATH,
    value=lowest_fare_id
)
print(lowest_fare_element)

<selenium.webdriver.remote.webelement.WebElement (session="4b3aec5844438e08e2f0ec2a2db1bf78", element="f82ae3bb-68e4-44c4-8df7-a79c8a9d29df")>


In [15]:
# sort the results by price, so that the lowest price is no longer hidden
sort_menu_element = driver.find_element(By.XPATH,value='//*[@id="searchResultsSortOptions"]')
action.move_to_element(sort_menu_element).perform()
time.sleep(button_press(0))

action.click(sort_menu_element).perform()
time.sleep(button_press(0))

# Move down to "Price (low to high)"
for x in range(0,2):
    action.send_keys(Keys.DOWN)
    time.sleep(button_press("keystroke"))

action.click(sort_menu_element).perform()
time.sleep(button_press(0))
    
# Click the button for the lowest flight option
action.move_to_element(lowest_fare_element).perform()
time.sleep(button_press(0))

action.click(lowest_fare_element).perform()
time.sleep(button_press(0))

In [16]:
# Identifying the list elements that contain flight and price information for return flights
# Get the html from the page
html_return = driver.page_source
html_soup_return = soup(html_return,'html.parser')

return_elements = html_soup_return.find_all("li",id=re.compile("slice1Flight\d"))

i = 0
for x in return_elements:
#     list_elements[i] = re.search("<li\s*>\w",str(x.prettify()))
    return_elements[i] = re.findall("<li.*?>",str(x.prettify()))[0]
    print(f"{i}: {return_elements[i]}")
    i += 1

0: <li class="flight-search-results js-moreflights" data-arrivaltime="06-26-2022 12:46:00" data-departuretime="06-26-2022 11:45:00" data-flightindex="1" data-lowestfare="597.00" data-tripprice="597.00" data-tripstops="0" data-triptime="181" id="slice1Flight1">
1: <li class="flight-search-results js-moreflights" data-arrivaltime="06-26-2022 13:55:00" data-departuretime="06-26-2022 12:54:00" data-flightindex="2" data-lowestfare="597.00" data-tripprice="597.00" data-tripstops="0" data-triptime="181" id="slice1Flight2">
2: <li class="flight-search-results js-moreflights" data-arrivaltime="06-26-2022 11:18:00" data-departuretime="06-26-2022 10:15:00" data-flightindex="3" data-lowestfare="597.00" data-tripprice="597.00" data-tripstops="0" data-triptime="183" id="slice1Flight3">
3: <li class="flight-search-results js-moreflights" data-arrivaltime="06-26-2022 15:48:00" data-departuretime="06-26-2022 14:46:00" data-flightindex="4" data-lowestfare="597.00" data-tripprice="597.00" data-tripstops=

In [17]:
#This tool solely used in early dev for identifying html elemnts that contain information


# button_elements = html_soup.findAll('button')
# i = 0
# for x in button_elements:
#     print(f"{i}: {x}")
#     i += 1

In [18]:
# driver.close()